# Local Controller Noise Analysis

Required penalty energy $E_p$ vs RMS noise amplitude for selected correlation times $\tau_X$:
- (a) Repetition code [[3,1,3]] with stabilizer penalty
- (b) Bacon-Shor [[4,1,1,2]] with gauge penalty

The simulation uses Ornstein-Uhlenbeck (OU) noise to model 1/f-like local control fluctuations.
Parameters are based on IBM dilution fridge conditions.

In [ ]:
import sys, os
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

sys.path.insert(0, os.path.dirname(os.getcwd()))
from qec_config import (
    QECConfig, BaconShorConfig, PlotConfig, SimulationParams,
    sigma_from_rms, ou_trace, simulate_control_noise_fidelity
)

PlotConfig.apply()

# Output directories
OUTPUT_DIR = Path('../figs/control_noise')
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
DATA_DIR = Path('../data/control_noise')
DATA_DIR.mkdir(parents=True, exist_ok=True)

print(f"Output: {OUTPUT_DIR}")
print(f"Data: {DATA_DIR}")

In [ ]:
# Simulation parameters
N_POINTS = 101  # Time discretization
N_REALIZATIONS = 10  # Noise realizations per point
TARGET_FID = 0.99  # Target fidelity

# Initialize configurations
rep = QECConfig(n_points=N_POINTS)
bs = BaconShorConfig(n_points=N_POINTS)

# Get key parameters
omega_max = rep.omega_max
T_max = rep.T_max

# RMS amplitude sweep (as fraction of omega_max)
RMS_FRACS = np.linspace(0.05, 0.50, 10)  # 5% to 50%
RMS_MHZ = RMS_FRACS * omega_max / (2 * np.pi * 1e6)  # Convert to MHz

# Correlation time values to plot (as fractions of T_max)
TAU_FRACS = np.array([0.10, 0.25, 0.50, 0.75])  # 10%, 25%, 50%, 75% of T_max
TAU_VALUES = TAU_FRACS * T_max  # Convert to seconds

# Ep grid to search (in rad/s)
EP_GRID = 2 * np.pi * np.linspace(10, 300, 30) * 1e6  # 10-300 MHz

print(f"omega_max = {omega_max/(2*np.pi*1e6):.1f} MHz")
print(f"T_max = {T_max*1e6:.1f} us")
print(f"RMS range: {RMS_FRACS[0]*100:.0f}% - {RMS_FRACS[-1]*100:.0f}% ({len(RMS_FRACS)} points)")
print(f"tau_X values: {TAU_FRACS*100} % of T_max")
print(f"Target fidelity: {TARGET_FID}")
print(f"Realizations: {N_REALIZATIONS}")

In [ ]:
# Simulate repetition code
print("Simulating Repetition Code [[3,1,3]]...")
print("="*60)

rep_Ep_required = np.zeros((len(TAU_VALUES), len(RMS_FRACS)))

for i, tau_X in enumerate(TAU_VALUES):
    tau_frac = tau_X / T_max
    print(f"\ntau_X = {tau_frac*100:.0f}% of T_max ({tau_X*1e6:.2f} us)")
    
    for j, rms_frac in enumerate(tqdm(RMS_FRACS, desc="  RMS sweep")):
        # Search for minimum Ep that achieves target fidelity
        found = False
        for Ep in EP_GRID:
            result = simulate_control_noise_fidelity(
                rep, Ep, rms_frac, tau_X,
                n_realizations=N_REALIZATIONS,
                n_steps=N_POINTS,
                seed=42 + i*100 + j,
                verbose=False
            )
            if result['fidelity_mean'] >= TARGET_FID:
                rep_Ep_required[i, j] = Ep
                found = True
                break
        
        if not found:
            rep_Ep_required[i, j] = np.nan

print("\n" + "="*60)

In [ ]:
# Simulate Bacon-Shor code
print("Simulating Bacon-Shor Code [[4,1,1,2]]...")
print("="*60)

bs_Ep_required = np.zeros((len(TAU_VALUES), len(RMS_FRACS)))

for i, tau_X in enumerate(TAU_VALUES):
    tau_frac = tau_X / T_max
    print(f"\ntau_X = {tau_frac*100:.0f}% of T_max ({tau_X*1e6:.2f} us)")
    
    for j, rms_frac in enumerate(tqdm(RMS_FRACS, desc="  RMS sweep")):
        # Search for minimum Ep that achieves target fidelity
        found = False
        for Ep in EP_GRID:
            result = simulate_control_noise_fidelity(
                bs, Ep, rms_frac, tau_X,
                n_realizations=N_REALIZATIONS,
                n_steps=N_POINTS,
                seed=1000 + i*100 + j,
                verbose=False
            )
            if result['fidelity_mean'] >= TARGET_FID:
                bs_Ep_required[i, j] = Ep
                found = True
                break
        
        if not found:
            bs_Ep_required[i, j] = np.nan

print("\n" + "="*60)

In [ ]:
# Create publication-quality figure
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5), sharey=True)

# Color map for tau values
colors = plt.cm.viridis(np.linspace(0.2, 0.9, len(TAU_VALUES)))
markers = ['o', 's', '^', 'D']

# Plot repetition code
for i, tau_X in enumerate(TAU_VALUES):
    tau_frac = tau_X / T_max
    label = rf'$\tau_X$ = {tau_frac*100:.0f}% $T_{{max}}$'
    Ep_MHz = rep_Ep_required[i] / (2 * np.pi * 1e6)
    ax1.plot(RMS_MHZ, Ep_MHz, f'{markers[i]}-', color=colors[i], 
             label=label, markersize=6, linewidth=2)

ax1.set_ylabel(r'Required $E_p$ [MHz]', fontsize=16)
ax1.set_title(r'(a) Repetition $[[3,1,3]]$', fontsize=18)
ax1.grid(True, alpha=0.3)
ax1.tick_params(labelsize=14)

# Plot Bacon-Shor code
for i, tau_X in enumerate(TAU_VALUES):
    tau_frac = tau_X / T_max
    label = rf'$\tau_X$ = {tau_frac*100:.0f}% $T_{{max}}$'
    Ep_MHz = bs_Ep_required[i] / (2 * np.pi * 1e6)
    ax2.plot(RMS_MHZ, Ep_MHz, f'{markers[i]}-', color=colors[i], 
             label=label, markersize=6, linewidth=2)

ax2.set_title(r'(b) Bacon-Shor $[[4,1,1,2]]$', fontsize=18)
ax2.grid(True, alpha=0.3)
ax2.tick_params(labelsize=14)

# Shared legend inside right plot
ax2.legend(title='Correlation Time', fontsize=10, title_fontsize=11, loc='upper left')

plt.tight_layout()
plt.subplots_adjust(bottom=0.15)

# Shared x label
fig.text(0.5, 0.02, r'RMS Noise Amplitude [MHz]', ha='center', fontsize=16)

plt.show()

In [ ]:
# Save figure in multiple formats
fig.savefig(OUTPUT_DIR / 'Ep_vs_RMS_comparison.pdf', bbox_inches='tight')
fig.savefig(OUTPUT_DIR / 'Ep_vs_RMS_comparison.svg', bbox_inches='tight')
fig.savefig(OUTPUT_DIR / 'Ep_vs_RMS_comparison.png', dpi=300, bbox_inches='tight')

# Save plot data for reproduction
plot_data = {
    # Sweep parameters
    'rms_fracs': RMS_FRACS,
    'rms_MHz': RMS_MHZ,
    'tau_fracs': TAU_FRACS,
    'tau_values_s': TAU_VALUES,
    'Ep_grid_rad': EP_GRID,
    # Results
    'rep_Ep_required_rad': rep_Ep_required,
    'rep_Ep_required_MHz': rep_Ep_required / (2 * np.pi * 1e6),
    'bs_Ep_required_rad': bs_Ep_required,
    'bs_Ep_required_MHz': bs_Ep_required / (2 * np.pi * 1e6),
    # Simulation parameters
    'omega_max_rad': omega_max,
    'omega_max_MHz': omega_max / (2 * np.pi * 1e6),
    'T_max_s': T_max,
    'T_max_us': T_max * 1e6,
    'target_fid': TARGET_FID,
    'n_realizations': N_REALIZATIONS,
    'n_points': N_POINTS,
}
np.savez_compressed(DATA_DIR / 'Ep_vs_RMS_data.npz', **plot_data)

print(f"Saved figures to {OUTPUT_DIR}")
print(f"Saved data to {DATA_DIR}")
print("\nFiles:")
for f in sorted(OUTPUT_DIR.glob('Ep_vs_RMS*')):
    print(f"  {f.name}")

In [ ]:
# Print summary table
print("\n" + "="*70)
print(f"SUMMARY: Required Ep [MHz] for F >= {TARGET_FID}")
print("="*70)

# Select RMS values to display
rms_display = [0.1, 0.2, 0.3, 0.4]
rms_indices = [np.argmin(np.abs(RMS_FRACS - r)) for r in rms_display]

print(f"\nRepetition Code [[3,1,3]]:")
print(f"{'tau_X':<20}", end="")
for r in rms_display:
    print(f"  RMS={r*100:3.0f}%", end="")
print()
print("-"*60)
for i, tau_X in enumerate(TAU_VALUES):
    tau_frac = tau_X / T_max
    print(f"{tau_frac*100:5.0f}% T_max       ", end="")
    for j in rms_indices:
        Ep_MHz = rep_Ep_required[i, j] / (2 * np.pi * 1e6)
        if np.isnan(Ep_MHz):
            print(f"     >300", end="")
        else:
            print(f"    {Ep_MHz:5.1f}", end="")
    print()

print(f"\nBacon-Shor Code [[4,1,1,2]]:")
print(f"{'tau_X':<20}", end="")
for r in rms_display:
    print(f"  RMS={r*100:3.0f}%", end="")
print()
print("-"*60)
for i, tau_X in enumerate(TAU_VALUES):
    tau_frac = tau_X / T_max
    print(f"{tau_frac*100:5.0f}% T_max       ", end="")
    for j in rms_indices:
        Ep_MHz = bs_Ep_required[i, j] / (2 * np.pi * 1e6)
        if np.isnan(Ep_MHz):
            print(f"     >300", end="")
        else:
            print(f"    {Ep_MHz:5.1f}", end="")
    print()

print("="*70)